In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
MAINDIR = "./gdrive/My Drive/EA-test/"

In [4]:
#Import of csv files
##### PlayerPersonalData #####
df_personal = pd.read_csv(MAINDIR + "PlayerPersonalData.csv", index_col=0)
df_personal.drop(columns=['Unnamed: 0.1'],inplace=True) # ELiminate additional index column

##### PlayerAttributeData #####
df_attribute = pd.read_csv(MAINDIR + "PlayerAttributeData.csv", index_col=0)
#Move to first position the ID column 
cols = df_attribute.columns.to_list()
cols2= cols[17:18] + cols[:17] + cols[18:]
df_attribute = df_attribute[cols2]
##### PlayerPlayingPositionData #####
df_PlayPosition = pd.read_csv(MAINDIR + "PlayerPlayingPositionData.csv", index_col=0)
#Move to first position the ID column 
cols = df_PlayPosition.columns.to_list()
cols[5:6]
cols2= cols[5:6] + cols[:5] + cols[6:]
df_PlayPosition = df_PlayPosition[cols2]

#Eliminate duplicate
print('\n','PlayerPersonalData.csv initial rows:', df_personal.shape[0])
print(' PlayerAttributeData.csv initial rows:', df_attribute.shape[0])
print(' PlayerPlayingPositionData.csv initial rows:', df_PlayPosition.shape[0],'\n')
df_personal.drop_duplicates(inplace=True)
df_attribute .drop_duplicates( inplace=True) 
df_PlayPosition.drop_duplicates( inplace=True)
print('\n','PlayerPersonalData.csv after drop duplicates rows:', df_personal.shape[0])
print(' PlayerAttributeData.csv after drop duplicates rows:', df_attribute.shape[0])
print(' PlayerPlayingPositionData.csv after drop duplicates rows:', df_PlayPosition.shape[0])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)



 PlayerPersonalData.csv initial rows: 17981
 PlayerAttributeData.csv initial rows: 17981
 PlayerPlayingPositionData.csv initial rows: 17981 


 PlayerPersonalData.csv after drop duplicates rows: 17929
 PlayerAttributeData.csv after drop duplicates rows: 17929
 PlayerPlayingPositionData.csv after drop duplicates rows: 17929


## PlayerPersonalData data quality analysis

In [5]:
#preview of data
df_personal.head()

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special
0,20801,Cristiano Ronaldo,32,https://cdn.sofifa.org/48/18/players/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,€95.5M,€565K,2228
1,158023,L. Messi,30,https://cdn.sofifa.org/48/18/players/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,93,93,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,€105M,€565K,2154
2,190871,Neymar,25,https://cdn.sofifa.org/48/18/players/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,94,Paris Saint-Germain,https://cdn.sofifa.org/24/18/teams/73.png,€123M,€280K,2100
3,176580,L. Suárez,30,https://cdn.sofifa.org/48/18/players/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,92,92,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,€97M,€510K,2291
4,167495,M. Neuer,31,https://cdn.sofifa.org/48/18/players/167495.png,Germany,https://cdn.sofifa.org/flags/21.png,92,92,FC Bayern Munich,https://cdn.sofifa.org/24/18/teams/21.png,€61M,€230K,1493


In [6]:
#With the info method we observe not null values, but 'Value' and 'Wage' are numeric values as object so we need to fix it to be numeric in case of be want used it in future calculations
df_personal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17929 entries, 0 to 17980
Data columns (total 13 columns):
ID             17929 non-null int64
Name           17929 non-null object
Age            17929 non-null int64
Photo          17929 non-null object
Nationality    17929 non-null object
Flag           17929 non-null object
Overall        17929 non-null int64
Potential      17929 non-null int64
Club           17681 non-null object
Club Logo      17929 non-null object
Value          17929 non-null object
Wage           17929 non-null object
Special        17929 non-null int64
dtypes: int64(5), object(8)
memory usage: 1.9+ MB


In [7]:
#We analize the unique values in columns and observe strings with currency simbols, abbreviation for million and thousand
#Here one example
df_personal.Value.unique()[:20]

array(['€95.5M', '€105M', '€123M', '€97M', '€61M', '€92M', '€64.5M',
       '€90.5M', '€79M', '€77M', '€52M', '€83M', '€59M', '€67.5M', '€57M',
       '€69.5M', '€66.5M', '€38M', '€4.5M', '€75M'], dtype=object)

In [0]:
# So we create a function to change string to numeric value and avoid error with values that are already of type integer or float
def evalif(n):
  if type(n) == str:
    if n[0] in ('€','$'):
      n = n[1:]
    if n[-1] == 'M':
      m = 1000000
      n = n[:-1]
    elif n[-1] == 'K':
      m = 1000
      n = n[:-1]
    else:
      m = 1

    return eval(n)*m
  # In case a no string value
  else:
    return n

In [0]:
#Now the function is applied in every column
cols = ['Value','Wage']

for c in cols:
  df_personal.loc[:,c] = df_personal[c].apply(evalif)

In [10]:
#And check the result to see the columns 'Value' as type float64 and 'Wage' as type int64
df_personal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17929 entries, 0 to 17980
Data columns (total 13 columns):
ID             17929 non-null int64
Name           17929 non-null object
Age            17929 non-null int64
Photo          17929 non-null object
Nationality    17929 non-null object
Flag           17929 non-null object
Overall        17929 non-null int64
Potential      17929 non-null int64
Club           17681 non-null object
Club Logo      17929 non-null object
Value          17929 non-null float64
Wage           17929 non-null int64
Special        17929 non-null int64
dtypes: float64(1), int64(6), object(6)
memory usage: 1.9+ MB


## PlayerAttributeData data quality analysis

In [11]:
#preview of data
df_attribute.head()

,ID,Acceleration,Aggression,Agility,Balance,Ball control,Composure,Crossing,Curve,Dribbling,Finishing,Free kick accuracy,GK diving,GK handling,GK kicking,GK positioning,GK reflexes,Heading accuracy,Interceptions,Jumping,Long passing,Long shots,Marking,Penalties,Positioning,Reactions,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys
0,20801,89,63,89,63,93,95,85,81,91,94,76,7,11,15,14,11,88,29,95,77,92,22,85,95,96,83,94,23,91,92,31,80,85,88
1,158023,92,48,90,95,95,96,77,89,97,95,90,6,11,15,14,8,71,22,68,87,88,13,74,93,95,88,85,26,87,73,28,59,90,85
2,190871,94,56,96,82,95,92,75,81,96,89,84,9,9,15,15,11,62,36,61,75,77,21,81,90,88,81,80,33,90,78,24,53,80,83
3,176580,88,78,86,60,91,83,77,86,86,94,84,27,25,31,33,37,77,41,69,64,86,30,85,92,93,83,87,38,77,89,45,80,84,88
4,167495,58,29,52,35,48,70,15,14,30,13,11,91,90,95,91,89,25,30,78,59,16,10,47,12,85,55,25,11,61,44,10,83,70,11


In [12]:
#With the info method we observe not null values, but numeric varibles are define as object
df_attribute.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17929 entries, 0 to 17980
Data columns (total 35 columns):
ID                    17929 non-null int64
Acceleration          17929 non-null object
Aggression            17929 non-null object
Agility               17929 non-null object
Balance               17929 non-null object
Ball control          17929 non-null object
Composure             17929 non-null object
Crossing              17929 non-null object
Curve                 17929 non-null object
Dribbling             17929 non-null object
Finishing             17929 non-null object
Free kick accuracy    17929 non-null object
GK diving             17929 non-null object
GK handling           17929 non-null object
GK kicking            17929 non-null object
GK positioning        17929 non-null object
GK reflexes           17929 non-null object
Heading accuracy      17929 non-null object
Interceptions         17929 non-null object
Jumping               17929 non-null object
Long passing

In [13]:
#We analize the unique values in columns and observe strings, and numbers in the same column
#Here one example
df_attribute['Free kick accuracy'].unique()

array(['76', '90', '84', '11', '19', '79', '62', '67', '83', '78', '77',
       '85', '72', '31', '13', '14', '75', '74', '61', '51', '53', '10',
       '82', '81', '49', '64', '87', '68', '43', '70', '66', '86', '59',
       '69', '45', '35', '39', '73', '47', '54', '71', '15', '8', '12',
       '26', '58', '91', '52', '63', '44', '28', '65', '57', '22', '27',
       '20', '30', '60', '56', '32', '50', '16', '42', '33', '89', '55',
       '37', '80', '48', '46', '38', '23', '36', '81+1', '18', '17', '25',
       '34', '40', '66-5', '24', '93', '73+1', '29', '88', '41', '21',
       '57+5', '60-4', '56+4', '7', '9', '60+8', '53+11', '63-7', '77-3',
       '4', '58+2', '69+1', '52+10', '62-3', '69-2', '65+1', '70+5',
       '39+2', '6', '58+6', '62-4', '65+5', '61+10', '71+4', '56+11',
       '64+1', '39+10', '55+21', '64+8', '70+3', '68+4', '70+17', '69+12',
       '65+24', '52+9', '70+21', '39-9', '66+30', '64-3', '70+4', '67-1',
       '39-6', '65+29', '56-2', '64-2', '36+4', '53+1',

In [0]:
#Now with the function evalif we 
cols = df_attribute.columns.to_list()

for c in cols[1:]:
  df_attribute.loc[:,c] =df_attribute[c].apply(evalif)

In [15]:
#And check the result to see all the columns as int64
df_attribute.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17929 entries, 0 to 17980
Data columns (total 35 columns):
ID                    17929 non-null int64
Acceleration          17929 non-null int64
Aggression            17929 non-null int64
Agility               17929 non-null int64
Balance               17929 non-null int64
Ball control          17929 non-null int64
Composure             17929 non-null int64
Crossing              17929 non-null int64
Curve                 17929 non-null int64
Dribbling             17929 non-null int64
Finishing             17929 non-null int64
Free kick accuracy    17929 non-null int64
GK diving             17929 non-null int64
GK handling           17929 non-null int64
GK kicking            17929 non-null int64
GK positioning        17929 non-null int64
GK reflexes           17929 non-null int64
Heading accuracy      17929 non-null int64
Interceptions         17929 non-null int64
Jumping               17929 non-null int64
Long passing          17929 non

## PlayerPlayingPositionData  data quality analysis

In [16]:
#preview of data
df_PlayPosition.head()

,ID,CAM,CB,CDM,CF,CM,LAM,LB,LCB,LCM,LDM,LF,LM,LS,LW,LWB,Preferred Positions,RAM,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,20801,89.0,53.0,62.0,91.0,82.0,89.0,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,ST LW,89.0,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,158023,92.0,45.0,59.0,92.0,84.0,92.0,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,RW,92.0,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,190871,88.0,46.0,59.0,88.0,79.0,88.0,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,LW,88.0,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,176580,87.0,58.0,65.0,88.0,80.0,87.0,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,ST,87.0,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
4,167495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
#In this case we do not need to do change in data, but the data has nan values
#And when a column has a null value all the columns, with the exception of 'ID' and 'Preferred Positions', have null values
print('## Rows without null values:',df_PlayPosition.dropna().shape[0], '\n')
df_PlayPosition.info()

## Rows without null values: 15905 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17929 entries, 0 to 17980
Data columns (total 28 columns):
ID                     17929 non-null int64
CAM                    15905 non-null float64
CB                     15905 non-null float64
CDM                    15905 non-null float64
CF                     15905 non-null float64
CM                     15905 non-null float64
LAM                    15905 non-null float64
LB                     15905 non-null float64
LCB                    15905 non-null float64
LCM                    15905 non-null float64
LDM                    15905 non-null float64
LF                     15905 non-null float64
LM                     15905 non-null float64
LS                     15905 non-null float64
LW                     15905 non-null float64
LWB                    15905 non-null float64
Preferred Positions    17929 non-null object
RAM                    15905 non-null float64
RB                     15905 

In [18]:
#Reviewing the data in 'Preferred Positions', we see with spaces at the end of the texts
df_PlayPosition['Preferred Positions'].unique()[:10]

array(['ST LW ', 'RW ', 'LW ', 'ST ', 'GK ', 'CDM CM ', 'CB ',
       'RM CM CAM ', 'RM LW ST LM ', 'ST CAM '], dtype=object)

In [19]:
#So we proceed to eliminate the white spaces at the begining or end of the texts
df_PlayPosition['Preferred Positions'] = df_PlayPosition['Preferred Positions'].str.rstrip()
df_PlayPosition['Preferred Positions'] = df_PlayPosition['Preferred Positions'].str.lstrip()
df_PlayPosition['Preferred Positions'].unique()[:10]

array(['ST LW', 'RW', 'LW', 'ST', 'GK', 'CDM CM', 'CB', 'RM CM CAM',
       'RM LW ST LM', 'ST CAM'], dtype=object)

In [20]:
#In the case of of null values, all correspond to the value 'GK' in 'Preferred Positions'
print('\'Preferred Positions\' values with nan in row:',df_PlayPosition[df_PlayPosition.CAM.isna()]['Preferred Positions'].unique(),'\n')
print('\'GK\' values in \'Preferred Positions\'  without nan:',sum(df_PlayPosition[~df_PlayPosition.CAM.isna()]['Preferred Positions'].unique() == 'GK'))

'Preferred Positions' values with nan in row: ['GK'] 

'GK' values in 'Preferred Positions'  without nan: 0


In [0]:
#So we know than in case of use this columns for calculation, we can filter by 'GK' to avoid errors or miscalculation
#And we can replace nan values for zero
df_PlayPosition.fillna(0,inplace=True)

## Join of the 3 data frames

In [22]:
#We check for any repeticion in names of column, knowing that ID column is in the three data frames
print('Unique names of columns:',len(set(df_personal.columns.to_list() + df_attribute.columns.to_list() + df_PlayPosition.columns.to_list())),'\n')
print('Total number of columns less the 2 ID columns repeated:',len(df_personal.columns) + len(df_attribute.columns) + len(df_PlayPosition.columns) -2,'\n')

Unique names of columns: 74 

Total number of columns less the 2 ID columns repeated: 74 



We get 74 in both cases, so there is no repetition for other column name

In [23]:
#We join the three tables, and we have the same number of rows and 74 columns
df = df_personal.merge(df_attribute, how="inner")
df = df.merge(df_PlayPosition, how="inner")
df.shape

(17929, 74)

In [0]:
#We will create a table to register the original dataframe for every column
df_obj = [df_personal.iloc[:,1:], df_attribute.iloc[:,1:], df_PlayPosition.iloc[:,1:]]
df_names =['PlayerPersonalData','PlayerAttributeData','PlayerPlayingPositionData']
clm_df = [('ID','All csv tables')]
for d, n in zip(df_obj,df_names):
  clm_df += list(zip(d.columns.to_list(),len(d.columns)*[n]))

In [0]:
 df_clm =pd.DataFrame(clm_df, columns=['column_name','table'])

In [0]:
#Save data frames as csv
df.to_csv(MAINDIR + 'CompleteCSVDataset.csv',index=False)
df_clm.to_csv(MAINDIR + 'ColumnDataIndex.csv')